In [1]:
import tensorflow as tf
import numpy as np
from utils import *
import os
import pickle
from cifar10_input import *
from cifar10_models import *

tf.logging.set_verbosity(tf.logging.INFO)

#### SET THIS BEFORE RUNNING
MODEL_NAME = "cifar10/cifar10_scm_001"
DISCRIMINATOR_NAME = "cifar10/cifar10_scm_001_discriminator_wdropout"
try:
    os.makedirs(CHECKPOINT.format(MODEL_NAME))
except FileExistsError:
    pass

class BundledData(InitializableMixin):
    def __init__(self, train_in, train_out, test_in, test_out):
        with tf.name_scope("data_splitter"):
            self.use_test_data = tf.Variable(0, name="input_test_vs_train", trainable=False)
            use_test_data = tf.cast(self.use_test_data, tf.bool)
            self.input = tf.cond(use_test_data, lambda: test_in, lambda: train_in)
            self.input.set_shape(test_in.shape)
            self.output = tf.cond(use_test_data, lambda: test_out, lambda: train_out)
            self.output.set_shape(test_out.shape)
           
        
tf.reset_default_graph()
train_files, test_files = maybe_download_and_extract()
train_data = Cifar10Record(train_files)
test_data = Cifar10Record(test_files)
data = BundledData(train_data.image, train_data.label, test_data.image, test_data.label)

model = Cifar10ShallowConvolutionalModel(data.input, data.output, trainable=False)
model = Cifar10ShallowConvolutionalModel(data.input, data.output, trainable=False)

try:
    checkpoints = sorted(["{}/{}".format(ROOT.format(MODEL_NAME),x[:-5]) for x in os.listdir(ROOT.format(MODEL_NAME)) if x.endswith(".meta")], 
                     key=lambda x: int(x.split("-")[-1]))
except FileNotFoundError:
    checkpoints = []
model.saver.recover_last_checkpoints(checkpoints)
try: latest_checkpoint = model.saver.last_checkpoints[-4]
except: latest_checkpoint = None

In [2]:
class Discriminator(TrainableMixin):
    def __init__(self, model, batch_size, summaries=True): 
        self.model = model
        self.batch_size = batch_size
        self._init_graph(summaries)
        self._init_saver()

    def _init_graph(self, summaries=True):
        with tf.name_scope("discriminator"):
            
            with tf.name_scope("meta_variables"):
                self.global_step = tf.Variable(0, name="global_step", trainable=False)
                self.learning_rate = tf.Variable(0.001, name="learning_rate", trainable=False)

            with tf.name_scope("layer_000_input"):
                self.input_dim = list(np.multiply(2, self.model.output_dim))
                self.output_dim = [2]
                self.use_test_data = tf.Variable(1, trainable=False)
                self.y_ = tf.one_hot(self.use_test_data, 2, on_value=1.0, off_value=0.0)
                self.y_ = tf.reshape(self.y_, [1] + self.output_dim)
                self.y_ = tf.tile(self.y_,[self.batch_size,1])
                self.x = tf.concat((self.model.proba, self.model.y_),1)
                layer_000_out_dim = self.input_dim
                
            with tf.name_scope("layer_001_fc"):
                layer_001_in_dim = layer_000_out_dim
                layer_001_out_dim = [100]
                self.W_001 = tf.Variable(tf.random_normal(layer_001_in_dim + layer_001_out_dim,0,0.1), name="weights")
                self.b_001 = tf.Variable(tf.random_normal(layer_001_out_dim,0,0.1), name="bias")
                self.logits_001 = tf.add(tf.matmul(self.x,self.W_001, name="mul_weights"), self.b_001, name="add_bias")
                self.bent_001 = tf.nn.relu(self.logits_001, name="relu")
                self.dropout_rate_001 = tf.Variable(2.0, name="dropout_rate", trainable=False)
                self.dropout_001 = tf.nn.dropout(self.bent_001, 1-self.dropout_rate_001, name="dropout")
                
            with tf.name_scope("layer_002_fc"):
                layer_002_in_dim = layer_001_out_dim
                layer_002_out_dim = [10]
                self.W_002 = tf.Variable(tf.random_normal(layer_002_in_dim + layer_002_out_dim,0,0.1), name="weights")
                self.b_002 = tf.Variable(tf.random_normal(layer_002_out_dim,0,0.1), name="bias")
                self.logits_002 = tf.add(tf.matmul(self.dropout_001,self.W_002, name="mul_weights"), self.b_002, name="add_bias")
                self.bent_002 = tf.nn.relu(self.logits_002, name="relu")
                self.dropout_rate_002 = tf.Variable(1.0, name="dropout_rate", trainable=False)
                self.dropout_002 = tf.nn.dropout(self.bent_002, 1-self.dropout_rate_002, name="dropout")

            with tf.name_scope("layer_003_fc"):
                layer_003_in_dim = layer_002_out_dim
                layer_003_out_dim = self.output_dim
                self.W_003 = tf.Variable(tf.random_normal(layer_003_in_dim + layer_003_out_dim,0,0.1), name="weights")
                self.b_003 = tf.Variable(tf.random_normal(layer_003_out_dim,0,0.1), name="bias")
                self.logits_003 = tf.add(tf.matmul(self.dropout_002,self.W_003, name="mul_weights"), self.b_003, name="add_bias")
                
            with tf.name_scope("outputs"):
                self.logits = self.logits_003
                self.proba = tf.nn.softmax(self.logits)
                self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y_, logits=self.logits))
                
            with tf.name_scope("summaries"):
                self.correct_prediction = tf.equal(tf.argmax(self.logits,1), tf.argmax(self.y_,1))
                self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32), name="accuracy")
                self.summaries = {}
                if summaries:
                    self.summaries = {
                        "batch acc.": tf.summary.scalar("batch_acc", self.accuracy)
                    }

            with tf.name_scope("training"): 
                self.optimizer = tf.train.GradientDescentOptimizer(self.learning_rate, name="optimizer")
                self.grads = self.optimizer.compute_gradients(self.loss)
                self.grad_application = self.optimizer.apply_gradients(self.grads, global_step = self.global_step)
                with tf.control_dependencies([self.grad_application] + list(self.summaries.values())):
                    self.train_step = tf.no_op(name="train_step")


In [3]:
discriminator = Discriminator(model, 100)

try:
    discriminator_checkpoints = sorted(["{}/{}".format(ROOT.format(DISCRIMINATOR_NAME),x[:-5]) for x in os.listdir(ROOT.format(DISCRIMINATOR_NAME)) if x.endswith(".meta")], 
                     key=lambda x: int(x.split("-")[-1]))
except FileNotFoundError:
    discriminator_checkpoints = []
discriminator.saver.recover_last_checkpoints(discriminator_checkpoints)
try: discriminator_latest_checkpoint = discriminator.saver.last_checkpoints[-13]
except: discriminator_latest_checkpoint = None

In [4]:
discriminator_checkpoints

['../models/cifar10/cifar10_scm_001_discriminator_wdropout/model.ckpt-500',
 '../models/cifar10/cifar10_scm_001_discriminator_wdropout/model.ckpt-1000',
 '../models/cifar10/cifar10_scm_001_discriminator_wdropout/model.ckpt-1500',
 '../models/cifar10/cifar10_scm_001_discriminator_wdropout/model.ckpt-2000',
 '../models/cifar10/cifar10_scm_001_discriminator_wdropout/model.ckpt-2500',
 '../models/cifar10/cifar10_scm_001_discriminator_wdropout/model.ckpt-3000',
 '../models/cifar10/cifar10_scm_001_discriminator_wdropout/model.ckpt-3500',
 '../models/cifar10/cifar10_scm_001_discriminator_wdropout/model.ckpt-4000',
 '../models/cifar10/cifar10_scm_001_discriminator_wdropout/model.ckpt-4500',
 '../models/cifar10/cifar10_scm_001_discriminator_wdropout/model.ckpt-5000',
 '../models/cifar10/cifar10_scm_001_discriminator_wdropout/model.ckpt-5500',
 '../models/cifar10/cifar10_scm_001_discriminator_wdropout/model.ckpt-6000',
 '../models/cifar10/cifar10_scm_001_discriminator_wdropout/model.ckpt-6500',


In [5]:
save_rate = 500
with tf.Session() as sess:
    model._restore_or_init(sess, latest_checkpoint)
    data._initialize(sess)
    discriminator._restore_or_init(sess, None)
    tf.assign(discriminator.dropout_rate_002, 0.5)
    tf.assign(discriminator.dropout_rate_001, 0.5)
    
    try:
        while True:
            sess.run(discriminator.train_step)
            global_step = discriminator.global_step.eval()
            use_test_data = global_step%2
            data.use_test_data.assign(use_test_data)
            discriminator.use_test_data.assign(use_test_data)
            if not global_step%save_rate:
                print("Saving to: ", CHECKPOINT.format(DISCRIMINATOR_NAME) + "_" + str(global_step))
                discriminator.saver.save(sess, CHECKPOINT.format(DISCRIMINATOR_NAME), global_step=global_step)
    except KeyboardInterrupt:
        pass


INFO:tensorflow:Restoring parameters from ../models/cifar10/cifar10_scm_001/model.ckpt-358500
Saving to:  ../models/cifar10/cifar10_scm_001_discriminator_wdropout/model.ckpt_500
INFO:tensorflow:../models/cifar10/cifar10_scm_001_discriminator_wdropout/model.ckpt-500 is not in all_model_checkpoint_paths. Manually adding it.
Saving to:  ../models/cifar10/cifar10_scm_001_discriminator_wdropout/model.ckpt_1000
INFO:tensorflow:../models/cifar10/cifar10_scm_001_discriminator_wdropout/model.ckpt-1000 is not in all_model_checkpoint_paths. Manually adding it.
Saving to:  ../models/cifar10/cifar10_scm_001_discriminator_wdropout/model.ckpt_1500
INFO:tensorflow:../models/cifar10/cifar10_scm_001_discriminator_wdropout/model.ckpt-1500 is not in all_model_checkpoint_paths. Manually adding it.
Saving to:  ../models/cifar10/cifar10_scm_001_discriminator_wdropout/model.ckpt_2000
INFO:tensorflow:../models/cifar10/cifar10_scm_001_discriminator_wdropout/model.ckpt-2000 is not in all_model_checkpoint_paths. 

Saving to:  ../models/cifar10/cifar10_scm_001_discriminator_wdropout/model.ckpt_18000
INFO:tensorflow:../models/cifar10/cifar10_scm_001_discriminator_wdropout/model.ckpt-18000 is not in all_model_checkpoint_paths. Manually adding it.
Saving to:  ../models/cifar10/cifar10_scm_001_discriminator_wdropout/model.ckpt_18500
INFO:tensorflow:../models/cifar10/cifar10_scm_001_discriminator_wdropout/model.ckpt-18500 is not in all_model_checkpoint_paths. Manually adding it.
